In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import os

In [2]:
data_dir = "C:\\data"
xls_path = os.path.join(data_dir,"company_list.xls")

In [7]:
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]


In [9]:
company_list = df['회사명']

In [15]:
from collections import defaultdict
company_dict = defaultdict(list)

In [25]:
# html = urlopen("https://www.amazon.com/books-used-books-textbooks/b?ie=UTF8&node=283155")  
html = urlopen("https://biz.chosun.com/")
bsObject = BeautifulSoup(html, "html.parser") 
bsObject_lxml = BeautifulSoup(html, 'lxml', from_encoding='utf-8')

# print(bsObject.prettify())

In [26]:
for i,meta in enumerate(bsObject_lsml.find_all('dl',class="news_item")):
    print(i,meta)

SyntaxError: invalid syntax (<ipython-input-26-c4b3a7401f36>, line 1)

In [31]:
for object in bsObject:
    print(object)

AttributeError: 'NavigableString' object has no attribute 'keys'

In [27]:
for link in bsObject.body.find_all('a'):
    print(link.text.strip(),link.get('href'))